# 项目3.4-神经网络压缩（权重量化）

## 友情提示
同学们可以前往课程作业区先行动手尝试 ！！！

## 项目描述

权重量化：减少模型权重的精度，降低模型的运算量和存储空间。        

## 数据集介绍
无

## 项目要求

权重量化: 用更好的方式来表现model中的参数，以此降低运算量/消耗容量。

## 数据准备

无

## 环境配置/安装

无

## 简介


Compression有很多种门派，在这裡我们会介绍上课出现过的其中四种，分别是:

* 知识蒸馏 Knowledge Distillation
* 网路剪枝 Network Pruning
* 用少量参数来做CNN Architecture Design
* 参数量化 Weight Quantization

在这个notebook中我们会介绍非常简单的Weight Quantization，
而我们有提供已经做完Knowledge Distillation的小model来做Quantization。

* Model架构 / Architecute Design在同目录中的hw7_Architecture_Design.ipynb。
* 已经train好的小model(“work/stu_net.pdparams”, 0.99M)
  * 参数为 base=16, width_mult=1 (default)


## 权重量化
![](https://ai-studio-static-online.cdn.bcebos.com/03573270bf4e4bb0a913018e11c1fa366407d6408b5142a5be96e82beaadb481)


我们这边会示范如何实作第一条: Using less bits to represent a value。

## 好的Quantization很重要。
这边提供一些TA的数据供各位参考。

|bit|state_dict size|accuracy|
|-|-|-|
|32|1047430 Bytes|0.81315|
|16|522958 Bytes|0.81347|
|8|268472 Bytes|0.80791|
|7|268472 Bytes|0.80791|


## 运算消耗
根据[paddle的官方手册](https://www.paddlepaddle.org.cn/documentation/docs/zh/2.0-rc/api/paddle/tensor/creation/Tensor_cn.html#tensor)，我们知道paddle.Tensor预设是32-bit，也就是佔了4byte的空间，而Tensor系列最低可以容忍的是16-bit。

为了方便操作，我们之后会将state_dict转成numpy array做事。
因此我们可以先看看numpy有什么样的type可以使用。
![](https://i.imgur.com/3N7tiEc.png)
而我们发现numpy最低有float16可以使用，因此我们可以直接靠转型将32-bit的tensor转换成16-bit的ndarray存起来。

# 导入权重

下载我们已经train好的小model的state_dict进行测试。

In [1]:
import os
import paddle

print(f"\noriginal cost: {os.stat('work/stu_net.pdparams').st_size} bytes.")
params = paddle.load('work/stu_net.pdparams')


original cost: 1583801 bytes.


# 32-bit 张量 -> 16-bit 

In [2]:
import numpy as np
import pickle

def encode16(params, fname):
    '''将params压缩成16-bit后输出到fname。

    Args:
      params: model的state_dict。
      fname: 压缩后输出的档名。
    '''

    custom_dict = {}
    for (name, param) in params.items():
        param = np.float64(param)
        # 有些东西不属于ndarray，只是一个数字，这个时候我们就不用压缩。
        if type(param) == np.ndarray:
            custom_dict[name] = np.float16(param)
        else:
            custom_dict[name] = param

    pickle.dump(custom_dict, open(fname, 'wb'))


def decode16(fname):
    '''从fname读取各个params，将其从16-bit还原回paddle.tensor后存进state_dict内。

    Args:
      fname: 压缩后的档名。
    '''

    params = pickle.load(open(fname, 'rb'))
    custom_dict = {}
    for (name, param) in params.items():
        param = paddle.Tensor(param)
        custom_dict[name] = param

    return custom_dict


encode16(params, '16_bit_model.pdparams')
print(f"16-bit cost: {os.stat('16_bit_model.pdparams').st_size} bytes.")

16-bit cost: 522254 bytes.


# 32-bit 张量 -> 8-bit (可选的)

这边提供转成8-bit的方法，仅供大家参考。
因为没有8-bit的float，所以我们先对每个weight记录最小值和最大值，进行min-max正规化后乘上$2^8-1$在四舍五入，就可以用np.uint8存取了。

$W' = round(\frac{W - \min(W)}{\max(W) - \min(W)} \times (2^8 - 1)$)



> 至于能不能转成更低的形式，例如4-bit呢? 当然可以，待你实作。

In [3]:
def encode8(params, fname):
    custom_dict = {}
    for (name, param) in params.items():
        param = np.float64(param)
        if type(param) == np.ndarray:
            min_val = np.min(param)
            max_val = np.max(param)
            param = np.round((param - min_val) / (max_val - min_val) * 255)
            param = np.uint8(param)
            custom_dict[name] = (min_val, max_val, param)
        else:
            custom_dict[name] = param

    pickle.dump(custom_dict, open(fname, 'wb'))


def decode8(fname):
    params = pickle.load(open(fname, 'rb'))
    custom_dict = {}
    for (name, param) in params.items():
        if type(param) == tuple:
            min_val, max_val, param = param
            param = np.float64(param)
            param = (param / 255 * (max_val - min_val)) + min_val
            param = paddle.Tensor(param)
        else:
            param = paddle.Tensor(param)

        custom_dict[name] = param

    return custom_dict

encode8(params, '8_bit_model.pdparams')
print(f"8-bit cost: {os.stat('8_bit_model.pdparams').st_size} bytes.")

8-bit cost: 267855 bytes.
